In [1]:
import pandas as pd

In [2]:
data = []  # 用于存储所有行的列表

with open("data.txt", "r") as file:
    count = 0
    for line in file:
        count += 1
        if count % 10 == 2 or count % 10 == 4:  # 只在每隔10行中的第二行和第四行添加数据
            line = line.strip()  # 去除行末尾的换行符和空白字符
            data.append(line)  # 将每行添加到数据列表中

In [3]:
data

['T/V                N    NB     P     Q               Time                 Gflops',
 'WR00L2L2        1960    60     2     2               2.96             1.6971e+00',
 'T/V                N    NB     P     Q               Time                 Gflops',
 'WR00L2L4        1960    60     2     2               3.68             1.3660e+00',
 'T/V                N    NB     P     Q               Time                 Gflops',
 'WR00L2C2        1960    60     2     2               3.99             1.2603e+00',
 'T/V                N    NB     P     Q               Time                 Gflops',
 'WR00L2C4        1960    60     2     2               3.73             1.3474e+00',
 'T/V                N    NB     P     Q               Time                 Gflops',
 'WR00L2R2        1960    60     2     2               3.45             1.4582e+00',
 'T/V                N    NB     P     Q               Time                 Gflops',
 'WR00L2R4        1960    60     2     2               3.43      

In [4]:
header = data[0]  # 提取第一行作为表头
data = data[1::2]  # 选择偶数行作为数据

In [5]:
header_list = header.split()

In [6]:
split_data = [line.split() for line in data]

In [7]:
df = pd.DataFrame(split_data, columns=header_list)

In [8]:
# 将'Gflops'列转换为浮点数类型
df['Gflops'] = df['Gflops'].astype(float)

##### 保存为Excel文件
```df.to_excel('data.xlsx', index=False)```

In [10]:
df

,T/V,N,NB,P,Q,Time,Gflops
0,WR00L2L2,1960,60,2,2,2.96,1.69710
1,WR00L2L4,1960,60,2,2,3.68,1.36600
2,WR00L2C2,1960,60,2,2,3.99,1.26030
3,WR00L2C4,1960,60,2,2,3.73,1.34740
4,WR00L2R2,1960,60,2,2,3.45,1.45820
...,...,...,...,...,...,...,...
139,WR00R2L4,2048,80,4,1,5.38,1.06510
140,WR00R2C2,2048,80,4,1,5.78,0.99225
141,WR00R2C4,2048,80,4,1,6.31,0.90874
142,WR00R2R2,2048,80,4,1,5.94,0.96547


In [11]:
grouped_df = df.groupby(['N', 'NB', 'P', 'Q'])

In [12]:
avg_gflops = grouped_df['Gflops'].apply(lambda x: x.mean())

In [16]:
avg_gflops_df = avg_gflops.reset_index()

In [17]:
avg_gflops_df

,N,NB,P,Q,Gflops
0,1960,60,2,2,1.367706
1,1960,60,4,1,0.762098
2,1960,80,2,2,1.493839
3,1960,80,4,1,0.877724
4,2048,60,2,2,1.571744
5,2048,60,4,1,1.010337
6,2048,80,2,2,1.661311
7,2048,80,4,1,0.975026


#### 保存成Excel文件
```avg_gflops_df.to_excel('avg_gflops.xlsx', index=False)```